In [15]:
import pandas as pd
import numpy as np
from collections import defaultdict
from IPython.display import HTML, display
pd.set_option('display.max_columns', None)

In [16]:
#Weekly Stats

#Week 1
Week_1_Stats = pd.read_csv('Week_1_NFL_2025.csv', quotechar="'")

#Week 2
Week_2_Stats = pd.read_csv('Week_2_NFL_2025.csv', quotechar="'")

#Week 3
Week_3_Stats = pd.read_csv('Week_3_NFL_2025.csv', quotechar="'")

#Week 4
Week_4_Stats = pd.read_csv('Week_4_NFL_2025.csv', quotechar="'")

Dataframes = [Week_1_Stats, Week_2_Stats, Week_3_Stats, Week_4_Stats]

for i, df in enumerate(Dataframes):
    df.columns = df.columns.str.replace('"', '', regex=False)
    df['Rk'] = df['Rk'].str.replace('"', '', regex=False)
    df = df.drop(['Rk', 'Day', 'Date', 'Unnamed: 12', 'Opp', 'Result', 'Att', 'Att.1', 'Tgt', 'G#', 'Week','OffSnp'], axis=1)
    df = df.rename(columns={'1D': 'Rush1D', '1D.1': 'Rec1D', 'OffSnp.1': 'OffSnp', 'Att.2': 'PassAtt','TD': 'PassTD', 'Yds': 'PassYds', 'Y/A': 'PassY/A', 'Yds.1': 'SackYds', 'Succ%': 'PassSucc%', 'Att.3': 'RushAtt','TD.1': 'RushTD', 'Yds.2': 'RushYds', 'Y/A/1': 'RushY/A', 'Succ%.1': 'RushSucc%', 'Tgt.1': 'Tgt', 'Yds.3': 'RecYds', 'TD.2': 'RecTD', 'Succ%.2': 'RecSucc%'})

    Dataframes[i] = df

#print(Week_3_Stats.columns)

In [17]:
#NFL Schedule
Schedule = pd.read_csv('Schedule_2025.csv')
Schedule = Schedule.map(lambda x: x.replace('@', '') if isinstance(x, str) else x)

#Conform to Stathead Labels
Schedule = Schedule.map(lambda x: x.replace('GB', 'GNB') if isinstance(x, str) else x)
Schedule = Schedule.map(lambda x: x.replace('KC', 'KAN') if isinstance(x, str) else x)
Schedule = Schedule.map(lambda x: x.replace('LV', 'LVR') if isinstance(x, str) else x)
Schedule = Schedule.map(lambda x: x.replace('NO', 'NOR') if isinstance(x, str) else x)
Schedule = Schedule.map(lambda x: x.replace('NE', 'NWE') if isinstance(x, str) else x)
Schedule = Schedule.map(lambda x: x.replace('SF', 'SFO') if isinstance(x, str) else x)
Schedule = Schedule.map(lambda x: x.replace('TB', 'TAM') if isinstance(x, str) else x)

#print(Schedule)

In [18]:
#Combine Total Stats
Week = len(Dataframes)+1
Combined = pd.concat(Dataframes, ignore_index=True)
Numeric_Part = Combined.groupby('Player', as_index=False).sum(numeric_only=True)
Non_Numeric_Part = Combined.groupby('Player', as_index=False).first(numeric_only=False).drop(columns=Numeric_Part.columns[1:])
Total_Stats = pd.merge(Numeric_Part, Non_Numeric_Part, on='Player')
#print(Total_Stats.columns)

In [19]:
Usefulcolumns = ['Player', 'Team', 'Opp', 'Week', 'Pos.', 'G', 'PassAtt', 'PassAttStDev', 'Cmp', 'IndComp%', 'CompStDev', 'TeamComp%', 'PassYds', 'PassYdsStDev', 'PassYds%', 'PassTD', 'PassTDStDev', 'PassTD%','Tgt', 'TgtStDev','Rec', 'IndCatch%', 'CatStDev', 'TmCatch%', 'RecYds', 'RecYdsStDev', 'RecYds%', 'RecTD', 'RecTDStDev', 'RecTD%', 'RushAtt', 'RushStDev', 'Rush%', 'RushYds', 'RushYdsStDev', 'RushYds%', 'RushTD', 'RushTDStDev', 'RushTD%', ]
Useful = pd.DataFrame(columns=Usefulcolumns)
Opponentcolumns = ['CompAAV', 'PassYdsAAV', 'PassTDAAV', 'CatchAAV', 'RecYdsAAV', 'RecTDAAV', 'RushAAV', 'RushYdsAAV', 'RushTDAAV']
Opponents = pd.DataFrame(columns=Opponentcolumns)
#print(Useful)
#print(Opponents)

In [20]:
# Update Week for all rows (no need to do it every iteration)
Useful['Week'] = Week

# Pre-map Opponents for quick lookup
schedule_map = {row[0]: row[Week] for row in Schedule.itertuples(index=False)}

# Pre-fill columns from Total_Stats if they exist
for col in Useful.columns:
    if col in Total_Stats.columns:
        Useful[col] = Total_Stats[col]

# Initialize containers for computed values
stat_fields = [
    'PassAtt', 'Cmp', 'PassYds', 'PassTD', 'Tgt',
    'Rec', 'RecYds', 'RecTD', 'RushAtt', 'RushYds', 'RushTD'
]

# Create per-player stat collections
player_stats = defaultdict(lambda: defaultdict(list))
player_games_played = defaultdict(int)

# Precompute all stats from all DataFrames
for df in Dataframes:
    if 'Player' not in df.columns:
        continue

    for row in df.itertuples(index=False):
        player = getattr(row, 'Player')

        player_games_played[player] += 1

        for stat in stat_fields:
            if hasattr(row, stat):
                val = getattr(row, stat)
                if pd.notnull(val):
                    player_stats[player][stat].append(val)

# Now update Useful efficiently
for i, row in Useful.iterrows():
    player = row['Player']
    team = row['Team']

    # Opponent from map
    Useful.at[i, 'Opp'] = schedule_map.get(team, None)

    # Games played
    Useful.at[i, 'G'] = player_games_played.get(player, 0)

    # Standard Deviations
    for stat in stat_fields:
        values = player_stats[player].get(stat, [])
        if len(values) >= 2:
            stdev = np.std(values, ddof=1)
        else:
            stdev = 0

        # Map stat name to your column names in Useful
        stdev_col_map = {
            'PassAtt': 'PassAttStDev',
            'Cmp': 'CompStDev',
            'PassYds': 'PassYdsStDev',
            'PassTD': 'PassTDStDev',
            'Tgt': 'TgtStDev',
            'Rec': 'CatStDev',
            'RecYds': 'RecYdsStDev',
            'RecTD': 'RecTDStDev',
            'RushAtt': 'RushStDev',
            'RushYds': 'RushYdsStDev',
            'RushTD': 'RushTDStDev',
        }

        if stat in stdev_col_map:
            Useful.at[i, stdev_col_map[stat]] = stdev

# Update Individual Completion Percentage
Useful['IndComp%'] = np.where(
    Useful['PassAtt'] != 0,
    (Useful['Cmp'] / Useful['PassAtt']),
    0
)

# Update Individual Catch Percentage
Useful['IndCatch%'] = np.where(
    Useful['Tgt'] != 0,
    (Useful['Rec'] / Useful['Tgt']),
    0
)


In [21]:
# Collect totals ONLY for games the player played in

# Initialize a list to collect results
totals = []
teamtotals = []

# Loop through each player in Useful
for player in Useful['Player']:
    total_team_pass_att = 0  # Total sum across all DataFrames
    total_team_pass_yards = 0
    total_team_pass_td = 0
    total_team_rec = 0
    total_team_rec_yards = 0
    total_team_rec_td = 0
    total_team_rush_att = 0
    total_team_rush_yards = 0
    total_team_rush_td = 0

    for df in Dataframes:
        if player in df['Player'].values:
            # Get the player's team (assumes 1 team per player per df)
            player_team = df.loc[df['Player'] == player, 'Team'].iloc[0]
            latest_df_with_player = df
            print(latest_df_with_player.columns)

            # Sum Cmp for all players on the same team
            #player_row = latest_df_with_player[latest_df_with_player['Player'] == player]
            #team_total_PA = latest_df_with_player.loc[latest_df_with_player['Team'] == player_team, 'PassAtt'].sum()
            team_total_PA = df.loc[df['Team'] == player_team, 'PassAtt'].sum()
            team_total_PY = df.loc[df['Team'] == player_team, 'PassYds'].sum()
            team_total_PT = df.loc[df['Team'] == player_team, 'PassTD'].sum()
            team_total_R = df.loc[df['Team'] == player_team, 'Rec'].sum()
            team_total_RY = df.loc[df['Team'] == player_team, 'RecYds'].sum()
            team_total_RT = df.loc[df['Team'] == player_team, 'RecTD'].sum()
            team_total_Ru = df.loc[df['Team'] == player_team, 'RushAtt'].sum()
            team_total_RuY = df.loc[df['Team'] == player_team, 'RushYds'].sum()
            team_total_RuT = df.loc[df['Team'] == player_team, 'RushTD'].sum()

            total_team_pass_att += team_total_PA
            total_team_pass_yards += team_total_PY
            total_team_pass_td += team_total_PT
            total_team_rec += team_total_R
            total_team_rec_yards += team_total_RY
            total_team_rec_td += team_total_RT
            total_team_rush_att += team_total_Ru
            total_team_rush_yards += team_total_RuY
            total_team_rush_td += team_total_RuT

    # Save result
    totals.append({'Player': player, 'TeamTotalPassAtt': total_team_pass_att, 'TeamTotalPassYds': total_team_pass_yards, 'TeamTotalPassTD': total_team_pass_td, 'TeamTotalRec': total_team_rec, 'TeamTotalRecYds': total_team_rec_yards, 'TeamTotalRecTD': total_team_rec_td, 'TeamTotalRushAtt': total_team_rush_att, 'TeamTotalRushYds': total_team_rush_yards, 'TeamTotalRushTD': total_team_rush_td})

# Create the final result DataFrame
IndividualTotals = pd.DataFrame(totals)
#print(totals)
#print(IndividualTotals)

Index(['Player', 'Age', 'Team', 'Cmp', 'PassAtt', 'Inc', 'Cmp%', 'PassYds',
       'PassTD', 'Int', 'Pick6', 'TD%', 'Int%', 'Rate', 'Sk', 'SackYds', 'Sk%',
       'PassY/A', 'AY/A', 'ANY/A', 'Y/C', 'PassSucc%', 'RushAtt', 'RushYds',
       'Y/A.1', 'RushTD', 'Rush1D', 'RushSucc%', 'Tgt', 'Rec', 'RecYds', 'Y/R',
       'RecTD', 'Ctch%', 'Y/Tgt', 'Rec1D', 'RecSucc%', 'OffSnp', 'Off%',
       'DefSnp', 'Def%', 'STSnp', 'ST %', 'Pos.', '-9999'],
      dtype='object')
Index(['Player', 'Age', 'Team', 'Cmp', 'PassAtt', 'Inc', 'Cmp%', 'PassYds',
       'PassTD', 'Int', 'Pick6', 'TD%', 'Int%', 'Rate', 'Sk', 'SackYds', 'Sk%',
       'PassY/A', 'AY/A', 'ANY/A', 'Y/C', 'PassSucc%', 'RushAtt', 'RushYds',
       'Y/A.1', 'RushTD', 'Rush1D', 'RushSucc%', 'Tgt', 'Rec', 'RecYds', 'Y/R',
       'RecTD', 'Ctch%', 'Y/Tgt', 'Rec1D', 'RecSucc%', 'OffSnp', 'Off%',
       'DefSnp', 'Def%', 'STSnp', 'ST %', 'Pos.', '-9999'],
      dtype='object')
Index(['Player', 'Age', 'Team', 'Cmp', 'PassAtt', 'Inc', 'Cmp%',

In [22]:
# Update Team Comp Percentage
Useful['TeamComp%'] = np.where(
    IndividualTotals['TeamTotalPassAtt'] != 0,
    (Useful['Cmp'] / IndividualTotals['TeamTotalPassAtt']),
    0
)

# Update Team Pass Yards Percentage
Useful['PassYds%'] = np.where(
    IndividualTotals['TeamTotalPassYds'] != 0,
    (Useful['PassYds'] / IndividualTotals['TeamTotalPassYds']),
    0
)

# Update Team Pass TD Percentage
Useful['PassTD%'] = np.where(
    IndividualTotals['TeamTotalPassTD'] != 0,
    (Useful['PassTD'] / IndividualTotals['TeamTotalPassTD']),
    0
)

# Update Team Catch Percentage
Useful['TmCatch%'] = np.where(
    IndividualTotals['TeamTotalRec'] != 0,
    (Useful['Rec'] / IndividualTotals['TeamTotalRec']),
    0
)

# Update Team Receiving Yards Percentage
Useful['RecYds%'] = np.where(
    IndividualTotals['TeamTotalRecYds'] != 0,
    (Useful['RecYds'] / IndividualTotals['TeamTotalRecYds']),
    0
)

# Update Team Receiving TD Percentage
Useful['RecTD%'] = np.where(
    IndividualTotals['TeamTotalRecTD'] != 0,
    (Useful['RecTD'] / IndividualTotals['TeamTotalRecTD']),
    0
)

# Update Team Rush Percentage
Useful['Rush%'] = np.where(
    IndividualTotals['TeamTotalRushAtt'] != 0,
    (Useful['RushAtt'] / IndividualTotals['TeamTotalRushAtt']),
    0
)

# Update Team Rush Yard Percentage
Useful['RushYds%'] = np.where(
    IndividualTotals['TeamTotalRushYds'] != 0,
    (Useful['RushYds'] / IndividualTotals['TeamTotalRushYds']),
    0
)

# Update Team Rush TD Percentage
Useful['RushTD%'] = np.where(
    IndividualTotals['TeamTotalRushTD'] != 0,
    (Useful['RushTD'] / IndividualTotals['TeamTotalRushTD']),
    0
)

print(len(Useful))

496


In [23]:

    # Initialize a list to collect results
teamtotals = []

# Loop through each player in Useful
for team in Schedule['Team']:
    total_team_completions = 0  # Total sum across all DataFrames
    total_team_pass_yards = 0
    total_team_pass_td = 0
    total_team_rec = 0
    total_team_rec_yards = 0
    total_team_rec_td = 0
    total_team_rush_att = 0
    total_team_rush_yards = 0
    total_team_rush_td = 0

    for df in Dataframes:
        #if player in df['Player'].values:
            # Get the player's team (assumes 1 team per player per df)
        #team = df.loc[df['Team'] == team, 'Team'].iloc[0]

        # Sum Cmp for all players on the same team
        team_total_C = df.loc[df['Team'] == team, 'Cmp'].sum()
        team_total_PY = df.loc[df['Team'] == team, 'PassYds'].sum()
        team_total_PT = df.loc[df['Team'] == team, 'PassTD'].sum()
        team_total_R = df.loc[df['Team'] == team, 'Rec'].sum()
        team_total_RY = df.loc[df['Team'] == team, 'RecYds'].sum()
        team_total_RT = df.loc[df['Team'] == team, 'RecTD'].sum()
        team_total_Ru = df.loc[df['Team'] == team, 'RushAtt'].sum()
        team_total_RuY = df.loc[df['Team'] == team, 'RushYds'].sum()
        team_total_RuT = df.loc[df['Team'] == team, 'RushTD'].sum()

        total_team_completions += team_total_C
        total_team_pass_yards += team_total_PY
        total_team_pass_td += team_total_PT
        total_team_rec += team_total_R
        total_team_rec_yards += team_total_RY
        total_team_rec_td += team_total_RT
        total_team_rush_att += team_total_Ru
        total_team_rush_yards += team_total_RuY
        total_team_rush_td += team_total_RuT

    # Save result
    teamtotals.append({'Team': team, 'CmpAAV': total_team_completions, 'PassYdsAAV': total_team_pass_yards, 'PassTDAAV': total_team_pass_td, 'RecAAV': total_team_rec, 'RecYdsAAV': total_team_rec_yards, 'RecTDAAV': total_team_rec_td, 'RushAttAAV': total_team_rush_att, 'RushYdsAAV': total_team_rush_yards, 'RushTDAAV': total_team_rush_td})

TeamTotals = pd.DataFrame(teamtotals)

for column in TeamTotals.columns[1:]:
    row_index=0
    
    if (df.iloc[row_index, :len(Dataframes)] == 'BYE').any():
        TeamTotals[column] = (TeamTotals[column] - TeamTotals[column].mean())/(len(Dataframes-1))
    else:
        TeamTotals[column] = (TeamTotals[column] - TeamTotals[column].mean())/len(Dataframes)
#print(TeamTotals.columns)



In [24]:
SuperFlexcolumns = ['Player', 'PPR', 'STD', 'PassYds', 'PassTD', 'Rec', 'RecYds', 'RecTD', 'RushAtt', 'RushYds', 'RushTD']
SuperFlex = pd.DataFrame(columns=SuperFlexcolumns)
Flexcolumns = ['Player', 'PPR', 'STD', 'PassYds', 'PassTD', 'Rec', 'RecYds', 'RecTD', 'RushAtt', 'RushYds', 'RushTD']
Flex = pd.DataFrame(columns=Flexcolumns)
WRcolumns = ['Player', 'PPR', 'STD', 'PassYds', 'PassTD', 'Rec', 'RecYds', 'RecTD', 'RushAtt', 'RushYds', 'RushTD']
WR = pd.DataFrame(columns=WRcolumns)
RBcolumns = ['Player', 'PPR', 'STD', 'PassYds', 'PassTD', 'Rec', 'RecYds', 'RecTD', 'RushAtt', 'RushYds', 'RushTD']
RB = pd.DataFrame(columns=RBcolumns)
TEcolumns = ['Player', 'PPR', 'STD', 'PassYds', 'PassTD', 'Rec', 'RecYds', 'RecTD', 'RushAtt', 'RushYds', 'RushTD']
TE = pd.DataFrame(columns=TEcolumns)
QBcolumns = ['Player', 'PPR', 'STD', 'PassYds', 'PassTD', 'Rec', 'RecYds', 'RecTD', 'RushAtt', 'RushYds', 'RushTD']
QB = pd.DataFrame(columns=QBcolumns)

#Populate Superflex with Player names
SuperFlex['Player'] = Useful['Player']

#Populate Flex with Player names
for i, row in Useful.iterrows():
    keywords = ['WR', 'RB', 'TE']
    if any(kw.lower() in row['Pos.'].lower() for kw in keywords):
        Flex.at[i, 'Player'] = row['Player']

#Populate WR with Player names
for i, row in Useful.iterrows():
    keywords = ['WR']
    if any(kw.lower() in row['Pos.'].lower() for kw in keywords):
        WR.at[i, 'Player'] = row['Player']

#Populate RB with Player names
for i, row in Useful.iterrows():
    keywords = ['RB']
    if any(kw.lower() in row['Pos.'].lower() for kw in keywords):
        RB.at[i, 'Player'] = row['Player']

#Populate TE with Player names
for i, row in Useful.iterrows():
    keywords = ['TE']
    if any(kw.lower() in row['Pos.'].lower() for kw in keywords):
        TE.at[i, 'Player'] = row['Player']

#Populate QB with Player names
for i, row in Useful.iterrows():
    keywords = ['QB']
    if any(kw.lower() in row['Pos.'].lower() for kw in keywords):
        QB.at[i, 'Player'] = row['Player']



In [25]:

n_simulations = 10000
team_stats = TeamTotals.set_index('Team').to_dict('index')

players = []

predictedrushes = []
predictedrushyards = []
predictedrushtds = []

predictedreceptions = []
predictedreceivingyards = []
predictedreceivingtds = []

predictedpassingyards = []
predictedpassingtds = []

pprs = []
stds = []

for i, row in Useful.iterrows():
    opp = row['Opp']
    rand = np.random.uniform(-2, 2, n_simulations)
    if opp == 'BYE':
        team = 'None'
        rushes = np.zeros(n_simulations)
        rushyards = np.zeros(n_simulations)
        rushtds= np.zeros(n_simulations)

        receptions = np.zeros(n_simulations)
        receivingyards = np.zeros(n_simulations)
        receivingtds = np.zeros(n_simulations)

        passingyards = np.zeros(n_simulations)
        passingtds = np.zeros(n_simulations)
        
    else:
        team = team_stats[opp]

        # Simulations
        rushes = (row['RushAtt'] / row['G']) + row['RushStDev'] * rand + team['RushAttAAV'] * row['Rush%']
        rushyards = (row['RushYds'] / row['G']) + row['RushYdsStDev'] * rand + team['RushYdsAAV'] * row['RushYds%']
        rushtds = (row['RushTD'] / row['G']) + row['RushTDStDev'] * rand + team['RushTDAAV'] * row['RushTD%']

        receptions = (row['Tgt'] / row['G']) * row['IndCatch%'] + row['TgtStDev'] * row['IndCatch%'] * rand + team['RecAAV'] * row['TmCatch%']
        receivingyards = (row['RecYds'] / row['G']) + row['RecYdsStDev'] * rand + team['RecYdsAAV'] * row['RecYds%']
        receivingtds = (row['RecTD'] / row['G']) + row['RecTDStDev'] * rand + team['RecTDAAV'] * row['RecTD%']

        passingyards = (row['PassYds'] / row['G']) + row['PassYdsStDev'] * rand + team['PassYdsAAV'] * row['PassYds%']
        passingtds = (row['PassTD'] / row['G']) + row['PassTDStDev'] * rand + team['PassTDAAV'] * row['PassTD%']

    players.append(row['Player'])

    predictedrushes.append(np.round(rushes.mean()).astype(int))
    predictedrushyards.append(np.round(rushyards.mean()).astype(int))
    predictedrushtds.append(np.round(rushtds.mean(),1))

    predictedreceptions.append(np.round(receptions.mean()).astype(int))
    predictedreceivingyards.append(np.round(receivingyards.mean()).astype(int))
    predictedreceivingtds.append(np.round(receivingtds.mean(),1))

    predictedpassingyards.append(np.round(passingyards.mean()).astype(int))
    predictedpassingtds.append(np.round(passingtds.mean(),1))
    

    # Fantasy scoring
    ppr = (
        np.round(rushyards.mean()).astype(int) / 10 +
        np.round(receivingyards.mean()).astype(int) / 10 +
        np.round(passingyards.mean()).astype(int) / 25 +
        np.round(receptions.mean()).astype(int) +
        (np.round(rushtds.mean(),1) + np.round(receivingtds.mean(),1)) * 6 +
        np.round(passingtds.mean(),1) * 4
    )
    pprs.append(ppr)

    std = (
        np.round(rushyards.mean()).astype(int) / 10 +
        np.round(receivingyards.mean()).astype(int) / 10 +
        np.round(passingyards.mean()).astype(int) / 25 +
        (np.round(rushtds.mean(),1) + np.round(receivingtds.mean(),1)) * 6 +
        np.round(passingtds.mean(),1) * 4
    )
    stds.append(std)

SuperFlex['RushAtt'] = predictedrushes
SuperFlex['RushYds'] = predictedrushyards
SuperFlex['RushTD'] = predictedrushtds
SuperFlex['Rec'] = predictedreceptions
SuperFlex['RecYds'] = predictedreceivingyards
SuperFlex['RecTD'] = predictedreceivingtds
SuperFlex['PassYds'] = predictedpassingyards
SuperFlex['PassTD'] = predictedpassingtds
SuperFlex['PPR'] = pprs
SuperFlex['STD'] = stds
SuperFlex.iloc[:, 1:3] = SuperFlex.iloc[:, 1:3].apply(pd.to_numeric).round(1)

for col in SuperFlex.columns:
    if col != 'Player' and col in Flex.columns:
        Flex[col] = Flex['Player'].map(SuperFlex.set_index('Player')[col])

for col in SuperFlex.columns:
    if col != 'Player' and col in WR.columns:
        WR[col] = WR['Player'].map(SuperFlex.set_index('Player')[col])

for col in SuperFlex.columns:
    if col != 'Player' and col in RB.columns:
        RB[col] = RB['Player'].map(SuperFlex.set_index('Player')[col])

for col in SuperFlex.columns:
    if col != 'Player' and col in TE.columns:
        TE[col] = TE['Player'].map(SuperFlex.set_index('Player')[col])

for col in SuperFlex.columns:
    if col != 'Player' and col in QB.columns:
        QB[col] = QB['Player'].map(SuperFlex.set_index('Player')[col])

In [26]:
All_DataFrames = {'SuperFlex': SuperFlex, 'Flex': Flex, 'WR': WR, 'RB': RB, 'TE': TE, 'QB': QB}
html_dict = {}

for name, df in All_DataFrames.items():

    html_string = df.to_html(classes='display', index=False).replace('class="dataframe display"', 'class="display"')

    # Full HTML file with sorting, search, and ALL rows shown
    html_script = f"""
    <!DOCTYPE html>
    <html>
    <head>
      <meta charset="utf-8">
      <title>SuperFlex Predictions</title>
      <script type="text/javascript"
      src="https://code.jquery.com/jquery-3.5.1.min.js"></script>
      <link rel="stylesheet" type="text/css"
      href="https://cdn.datatables.net/1.13.6/css/jquery.dataTables.min.css"/>
      <script type="text/javascript"
      src="https://cdn.datatables.net/1.13.6/js/jquery.dataTables.min.js"></script>
      <link rel="stylesheet" href="https://cdn.datatables.net/fixedheader/3.4.0/css/fixedHeader.dataTables.min.css">
      <script src="https://cdn.datatables.net/fixedheader/3.4.0/js/dataTables.fixedHeader.min.js"></script>

      <link rel="stylesheet" href="style.css">
    
    <title>Week 5 Predictions</title>

    </head>
    <body>

    <a href="index.html">Back to Home</a>
    <a href="SuperFlex.html">SuperFlex</a>
    <a href="Flex.html">Flex</a>
    <a href="WR.html">WR</a>
    <a href="RB.html">RB</a>
    <a href="TE.html">TE</a>
    <a href="QB.html">QB</a>

    <script>
    $(document).ready(function() {{
        var table = $('table.display').DataTable({{
            paging: true,
            searching: true,
            ordering: true,
            pageLength: -1,  // Show ALL rows
            order: [[1, 'desc']],  // Sort by PPR
            fixedHeader: true //scrollX: true
        }});
        table.draw();  // Force a redraw in case anything breaks
    }});
    </script>

    {html_string}

    </body>
    </html>
    """

    # Display in notebook (sortable)
    #display(HTML(SuperFlex_html_script))

    # Save to HTML file
    with open(f"{name}.html", "w") as f:
        f.write(html_script)

In [27]:
commit_msg = f"Adding data for Week {Week-1} and Producing predictions for {Week}"

!git add .
!git commit -m "{commit_msg}"
!git push

[main fef7b38] Adding data for Week 4 and Producing predictions for 5
 6 files changed, 848 insertions(+), 836 deletions(-)


To https://github.com/arahe1/SuperFlex_Football_Stats.git
   6ab7294..fef7b38  main -> main
